In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.autograd as autograd
device = 'cuda:3'
import torch
import numpy as np
# Set random seeds for reproducibility
#torch.manual_seed(1)
#np.random.seed(1)

class Problems():
    def dlX_disp(self):
        domain_xcoord = np.random.uniform(-self.nelx/(2*(self.nelm)),self.nelx/(2*(self.nelm)),(self.batch_size - self.dlX_fixed.shape[0] - self.dlX_force.shape[0],1))
        domain_ycoord = np.random.uniform(-self.nely/(2*(self.nelm)),self.nely/(2*(self.nelm)),(self.batch_size - self.dlX_fixed.shape[0] - self.dlX_force.shape[0],1))
        domain_coord = np.concatenate((domain_ycoord,domain_xcoord),axis = 1)
        coord = np.concatenate((self.dlX_fixed.cpu().detach().numpy(), self.dlX_force.cpu().detach().numpy()),axis = 0)
        coord = np.concatenate((coord, domain_coord),axis = 0)
        coord = torch.tensor(coord,dtype=torch.float32,requires_grad=True).to(device)
        return coord
        
    def dlX_disp1(self):
        '''
        domain_xcoord = np.random.uniform(-self.nelx/(2*(self.nelm)),self.nelx/(2*(self.nelm)),(self.batch_size - self.dlX_fixed.shape[0] - self.dlX_force.shape[0],1))
        domain_ycoord = np.random.uniform(-self.nely/(2*(self.nelm)),self.nely/(2*(self.nelm)),(self.batch_size - self.dlX_fixed.shape[0] - self.dlX_force.shape[0],1))
        domain_coord = np.concatenate((domain_ycoord,domain_xcoord),axis = 1)
        coord = np.concatenate((self.dlX_fixed.cpu().detach().numpy(), self.dlX_force.cpu().detach().numpy()),axis = 0)
        coord = np.concatenate((coord, domain_coord),axis = 0)
        coord = torch.tensor(coord,dtype=torch.float32,requires_grad=True).to(device)
        '''
        x = np.linspace(-self.nelx/(2*(self.nelm)),self.nelx/(2*(self.nelm)),num = 200)
        y = np.linspace(-self.nely/(2*(self.nelm)),self.nely/(2*(self.nelm)),num = 100)

        X, Y = np.meshgrid(x,y)
        Y = np.flipud(Y)

        data = np.hstack([Y.reshape(-1,1,order='F'), X.reshape(-1,1,order='F')])
        coord = torch.tensor(data,dtype=torch.float32,requires_grad=True).to(device)
        return coord

class Cantilever_Beam_2D(Problems):
    def __init__(self, nelx, nely, xid, yid, vf):

        # Initialize geometry parameters
        self.xid = xid
        self.yid = yid
        self.nelx = nelx
        self.nely = nely

        self.nele = self.nelx*self.nely
        self.nelm = max(self.nelx,self.nely)
        self.volfrac = vf
        self.E0 = 1
        self.nu = 0.3
        
        self.batch_size = 25000

        self.alpha_init = 1
        self.alpha_max = 100
        self.alpha_delta = 0.5
        self.penal = 3.0
        
        c_y, c_x=np.meshgrid(np.linspace(-(self.nely)/(2*self.nelm),(self.nely)/(2*self.nelm),self.nely),
                                                np.linspace(-(self.nelx)/(2*self.nelm),(self.nelx)/(2*self.nelm),self.nelx)
                                                ,indexing='ij')
        self.dlX = np.stack((c_y.reshape([-1]),c_x.reshape([-1])),axis = 1).reshape([-1,2])
        c_y, c_x=np.meshgrid(np.linspace(-(self.nely)/(2*self.nelm),(self.nely)/(2*self.nelm),2*self.nely),
                                                np.linspace(-(self.nelx)/(2*self.nelm),(self.nelx)/(2*self.nelm),2*self.nelx)
                                                ,indexing='ij')
        self.dlXSS = np.stack((c_y.reshape([-1]),c_x.reshape([-1])),axis = 1).reshape([-1,2])
        self.V = (np.max(self.dlX[:,0])-np.min(self.dlX[:,0]))*(np.max(self.dlX[:,1])-np.min(self.dlX[:,1]))

        #Problem boundary condition
        fixed_voxel = np.zeros((self.nely,self.nelx))
        fixed_voxel[:,0] = 1.0
        fixed_voxel = fixed_voxel.reshape([self.nele,1])
        dlX_fixed = self.dlX[np.where(fixed_voxel == 1.0)[0],:]

        F = 0.1
        self.F_vector = torch.tensor([[F], [0.0]], dtype=torch.float32).to(device)
        self.force_voxel = np.zeros((self.nely,self.nelx)) 
        self.force_voxel[yid,xid] = 1
        force_voxel = self.force_voxel.reshape([self.nele,1])
        dlX_force = self.dlX[np.where(force_voxel == 1)[0],:]

        self.dlX = torch.tensor(self.dlX, dtype=torch.float32,requires_grad=True).to(device)
        self.dlXSS = torch.tensor(self.dlXSS, dtype=torch.float32,requires_grad=True).to(device)
        self.dlX_fixed = torch.tensor(dlX_fixed, dtype=torch.float32,requires_grad=True).to(device)
        self.dlX_force = torch.tensor(dlX_force, dtype=torch.float32,requires_grad=True).to(device)

    def analytical_fixed_BC(self,u,coord):
        u = u*2*(1/(1+torch.exp(-20*(coord[:,1:2]+0.5))) - 0.5)
        return u


import torch.nn.functional as F

class TO_Net(nn.Module):
    def __init__(self):
        super(TO_Net, self).__init__()
        low_band = 0.0
        high_band = 35
        c_y, c_x=np.meshgrid(np.linspace([-high_band,low_band],[-low_band,high_band],10).reshape([-1]),
                                                    np.linspace([-high_band,low_band],[-low_band,high_band],10).reshape([-1])
                                                    ,indexing='ij')
        self.kernel1 = torch.tensor(np.stack((c_y.reshape([-1]),c_x.reshape([-1])),axis = 0),dtype=torch.float32).to(device)
        self.kernel1 = torch.nn.Parameter(self.kernel1.requires_grad_())
        self.weights1 = torch.zeros([self.kernel1.shape[1],1],dtype=torch.float32).to(device)
        self.weights1 = torch.nn.Parameter(self.weights1.requires_grad_())
            
    def forward(self, x):
        y = torch.sin(torch.matmul(x,1.0* self.kernel1 ) + torch.ones([1,self.kernel1.shape[1]]).to(device))
        y = torch.sigmoid(torch.matmul(y, self.weights1))
        return y
    
class Disp_Net(nn.Module):
    def __init__(self):
        super(Disp_Net, self).__init__()
        low_band = 0.0
        high_band = 35
        c_y, c_x=np.meshgrid(np.linspace([-high_band,low_band],[-low_band,high_band],10).reshape([-1]),
                                                    np.linspace([-high_band,low_band],[-low_band,high_band],10).reshape([-1]),
                                                    indexing='ij')
        self.kernel1 = torch.tensor(np.stack((c_y.reshape([-1]),c_x.reshape([-1])),axis = 0),dtype=torch.float32).to(device)
        #self.kernel1 = torch.nn.Parameter(self.kernel1.requires_grad_())
        #self.weights1 = torch.zeros([self.kernel1.shape[1],2],dtype=torch.float32).to(device)
        #self.weights1 = torch.nn.Parameter(self.weights1.requires_grad_())
    def forward(self, x):
        y = torch.sin(torch.matmul(x,1.0* self.kernel1 ) + torch.ones([1,self.kernel1.shape[1]]).to(device))
        #y = torch.matmul(y, self.weights1)
        return y
    
def cal_matrix(models,points,dlx_force,M):
    
    v_force = models(dlx_force).cpu().detach().numpy().reshape(-1,M)
           
    in_ = points.cpu().detach().numpy()
    u_x = np.zeros((in_.shape[0],M))
    u_y = np.zeros((in_.shape[0],M))
    u1 = np.zeros((in_.shape[0],M))
    temp_x=points[:,1:2].reshape(-1,1)
    u = models(points)*2*(1/(1+torch.exp(-20*(temp_x+0.5))) - 0.5)      
    values_u = u.cpu().detach().numpy()      
    u_value = values_u.reshape(-1,M)     
    grad_u_x = []
    grad_u_y = []
    for i in range(M):
        g_u = torch.autograd.grad(outputs=u[:,i], inputs=points,
                                grad_outputs=torch.ones_like(u[:,i]),
                                create_graph = True, retain_graph = True)[0]      
        grad_u_y.append(g_u[:,0].squeeze().cpu().detach().numpy())
        grad_u_x.append(g_u[:,1].squeeze().cpu().detach().numpy())      
    grad_u_x = np.array(grad_u_x)
    grad_u_x = grad_u_x.transpose(1,0).reshape(-1,M)        
    grad_u_y = np.array(grad_u_y)
    grad_u_y = grad_u_y.transpose(1,0).reshape(-1,M)
    u_x[:, :] = grad_u_x
    u_y[:, :] = grad_u_y
    u1[:, :] = u_value
    return u1,u_x,u_y,v_force

from torch.func import jacrev, vmap
def cal_matrix_fast(models, points, dlx_force, M):
    # points: [N, 3]，提前设置 requires_grad
    points = points.detach().clone().requires_grad_(True)

    # force
    v_force = models(dlx_force).detach().cpu().numpy().reshape(-1, M)
    
    # 计算 u
    

    def model_wrapped(p_single):
        # p_single: [3], 1个点
        out = models(p_single.unsqueeze(0))  # shape: [1, M]
        temp_x_single = p_single[1:2]  # shape: [1]
        mask_single = 2 * (1 / (1 + torch.exp(-20 * (temp_x_single + 0.5))) - 0.5)
        return (out.squeeze(0) * mask_single)  # shape: [M]

    # 得到 [N, M, 3]
    J = vmap(jacrev(model_wrapped))(points)

    # J: [N, M, 3]，按 dim=2 拆分
    grad_u_y = J[:, :, 0].detach().cpu().numpy()
    grad_u_x = J[:, :, 1].detach().cpu().numpy()

    t = 0 

    return  t, grad_u_x, grad_u_y, v_force

nelx = 60
nely = 20

#User defined load location
xid = 59
yid = 9

#User defined target volume fraction
vf = 0.5
problem= Cantilever_Beam_2D(nelx,nely,xid,yid,vf)
to_model= TO_Net().to(device)
disp_model_h = Disp_Net().to(device)
M = disp_model_h.kernel1.shape[1]

def oc(total_num,x,volfrac,dc,dv,g):
	l1=0
	l2=1e9
	move=0.2
	# reshape to perform vector operations
	xnew=np.zeros(total_num)
	while (l2-l1)/(l1+l2)>1e-3:
		lmid=0.5*(l2+l1)
		xnew[:]= np.maximum(0.0,np.maximum(x-move,np.minimum(1.0,np.minimum(x+move,x*np.sqrt(-dc/dv/lmid)))))
		gt=g+np.sum((dv*(xnew-x)))
		if gt>0 :
			l1=lmid
		else:
			l2=lmid
	return (xnew,gt)
def DensityFilter_init(rmin, nelx, nely):  
    nfilter = int(nelx * nely * ((2 * (np.ceil(rmin) - 1) + 1) ** 2))
    iH = np.zeros(nfilter)
    jH = np.zeros(nfilter)
    sH = np.zeros(nfilter)
    cc = 0
    for i in range(nelx):
        for j in range(nely):
            row = i * nely + j
            kk1 = int(np.maximum(i - (np.ceil(rmin) - 1), 0))
            kk2 = int(np.minimum(i + np.ceil(rmin), nelx))
            ll1 = int(np.maximum(j - (np.ceil(rmin) - 1), 0))
            ll2 = int(np.minimum(j + np.ceil(rmin), nely))
            for k in range(kk1, kk2):
                for l in range(ll1, ll2):
                    col = k * nely + l
                    fac = rmin - np.sqrt(((i - k) * (i - k) + (j - l) * (j - l)))
                    iH[cc] = row
                    jH[cc] = col
                    sH[cc] = np.maximum(0.0, fac)
                    cc = cc + 1
    # Finalize assembly and convert to csc format
    H = coo_matrix((sH, (iH, jH)), shape=(nelx * nely, nelx * nely)).tocsc()
    Hs = H.sum(1)
    return H, Hs

def pinnloss2(weights1, u_x, u_y, v_force, problem, xPhys_m):
    xPhys_m = xPhys_m.reshape(-1,1)
    ux = torch.matmul(u_x,weights1[:,1]).reshape(-1,1)
    uy = torch.matmul(u_y,weights1[:,1]).reshape(-1,1)
    vx = torch.matmul(u_x,weights1[:,0]).reshape(-1,1)
    vy = torch.matmul(u_y,weights1[:,0]).reshape(-1,1)
                                      
    eps11 = ux
    eps12 = 0.5 * uy + 0.5 * vx
    eps22 = vy

    youngs_modulus = 1000
    poissons_ratio = 0.3
    lame_mu = youngs_modulus / (2.0 * (1.0 + poissons_ratio))
    lame_lambda = youngs_modulus * poissons_ratio / (1.0 - poissons_ratio**2)
    trace_strain = eps11 + eps22
    squared_diagonal = eps11 * eps11 + eps22 * eps22
    energy = 0.5 * lame_lambda * trace_strain * trace_strain + lame_mu * (squared_diagonal + 2.0 * eps12 * eps12)
    energy = energy.reshape(-1,1)*(xPhys_m**3.0)
    energy_c = energy
    energy_ans =problem.V*torch.mean(energy)

    vforce = torch.matmul(v_force,weights1[:,0]).reshape(-1,1)
    #uforce = cp.dot(u_force,w[M:2*M]).reshape(-1,1)
    #rforce = cp.dot(r_force,w[2*M:]).reshape(-1,1)
    #bforce = cp.concatenate((vforce,uforce,rforce),axis=1)
    force_l = torch.mean(0.1*vforce)
    loss = (energy_ans - force_l)
    return loss,energy_c




In [2]:
import cupy as cp
def pinnloss(weights1, u_x, u_y, v_force, problem, xPhys_m):
    shape1 = M
    weights1 = cp.array(weights1)
    xPhys_m = cp.array(xPhys_m.reshape(-1,1))
    ux = cp.dot(u_x,weights1[shape1:]).reshape(-1,1)
    uy = cp.dot(u_y,weights1[shape1:]).reshape(-1,1)

    vx = cp.dot(u_x,weights1[:shape1]).reshape(-1,1)
    vy = cp.dot(u_y,weights1[:shape1]).reshape(-1,1)

                                   
    eps11 = ux
    eps12 = 0.5 * uy + 0.5 * vx
    eps22 = vy

    youngs_modulus = 1000
    poissons_ratio = 0.3
    lame_mu = youngs_modulus / (2.0 * (1.0 + poissons_ratio))
    lame_lambda = youngs_modulus * poissons_ratio / (1.0 - poissons_ratio**2)
    trace_strain = eps11 + eps22
    squared_diagonal = eps11 * eps11 + eps22 * eps22
    energy = 0.5 * lame_lambda * trace_strain * trace_strain + lame_mu * (squared_diagonal + 2.0 * eps12 * eps12)
    energy = energy.reshape(-1,1)*(xPhys_m**3.0)
    energy_ans =problem.V*cp.mean(energy)

    vforce = cp.dot(v_force,weights1[:shape1]).reshape(-1,1)
    force_l = cp.mean(0.1*vforce)
    loss = (energy_ans - force_l)
    return  cp.asnumpy(loss)

def pinnloss3(weights1, u_x, u_y, v_force, problem, xPhys_m):
    shape1 = M
    weights1 = cp.array(weights1)
    xPhys_m = cp.array(xPhys_m.reshape(-1,1))
    ux = cp.dot(u_x,weights1[shape1:]).reshape(-1,1)
    uy = cp.dot(u_y,weights1[shape1:]).reshape(-1,1)

    vx = cp.dot(u_x,weights1[:shape1]).reshape(-1,1)
    vy = cp.dot(u_y,weights1[:shape1]).reshape(-1,1)

                                   
    eps11 = ux
    eps12 = 0.5 * uy + 0.5 * vx
    eps22 = vy

    youngs_modulus = 1000
    poissons_ratio = 0.3
    lame_mu = youngs_modulus / (2.0 * (1.0 + poissons_ratio))
    lame_lambda = youngs_modulus * poissons_ratio / (1.0 - poissons_ratio**2)
    trace_strain = eps11 + eps22
    squared_diagonal = eps11 * eps11 + eps22 * eps22
    energy = 0.5 * lame_lambda * trace_strain * trace_strain + lame_mu * (squared_diagonal + 2.0 * eps12 * eps12)
    energy = energy.reshape(-1,1)*(xPhys_m**3.0)
    return  cp.asnumpy(energy)


def jac(weights1, u_x, u_y, v_force, problem, xPhys_m):

    w = cp.array(weights1)
    xPhys_m = xPhys_m.reshape(-1,1)
    xPhys_m = cp.array(xPhys_m)
    ux = cp.dot(u_x,w[M:2*M]).reshape(-1,1)
    uy = cp.dot(u_y,w[M:2*M]).reshape(-1,1)
   
    vx = cp.dot(u_x,w[:M]).reshape(-1,1)
    vy = cp.dot(u_y,w[:M]).reshape(-1,1)
    
 
                                        
    eps11 = ux
    eps12 = 0.5 * uy + 0.5 * vx
    eps22 = vy
   

    youngs_modulus = 1000
    poissons_ratio = 0.3
    lame_mu = youngs_modulus / (2.0 * (1.0 + poissons_ratio))
    lame_lambda = youngs_modulus * poissons_ratio / ((1.0 + poissons_ratio) * (1.0 - 2.0 * poissons_ratio))
    trace_strain = eps11 + eps22
    
    temp1 = (0.5 * lame_lambda * 2 * trace_strain * u_y + lame_mu * (2 * vy * u_y + 2 * 2 * eps12 * 0.5 * u_x))*(xPhys_m**3.0)
    loss_w1 = problem.V*cp.mean(temp1,axis=0).reshape(1,-1) - 0.1 * v_force

    temp2 = (0.5 * lame_lambda * 2 * trace_strain * u_x + lame_mu * (2 * ux * u_x+ 2 * 2 * eps12 * 0.5 * u_y ))*(xPhys_m**3.0)
    loss_w2 = problem.V*cp.mean(temp2,axis=0).reshape(1,-1)
    loss_w1 = cp.asnumpy(loss_w1)
    loss_w2 = cp.asnumpy(loss_w2)
    loss_w = np.concatenate((loss_w1,loss_w2),axis=1)
    return loss_w.squeeze()

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from IPython import display
import numpy as np
from scipy.optimize import minimize
from scipy.sparse import coo_matrix
class DMF_TONN():

    def __init__(self, problem, to_model, disp_model,solver='Adam',OC_use=False):
        self.problem = problem
        self.disp_model = disp_model
        self.to_model = to_model
        self.total_epoch = 0
        self.to_optimizer = optim.Adam(self.to_model.parameters(), lr=0.002)
        self.solver = solver
        self.x0 = np.random.random(M*2)*0.01
        self.OC_use = OC_use
        if self.OC_use:
            self.mse_loss = nn.MSELoss()
            self.H,self.Hs = DensityFilter_init(3, 200, 100)
            self.coord = problem.dlX_disp1()
        else:
            self.coord = problem.dlX_disp()
        _,u_x,u_y,v_force = cal_matrix_fast(self.disp_model, self.coord, self.problem.dlX_force, M)
        if solver=='Adam':
            self.u_x = torch.tensor(u_x,dtype=torch.float32).to(device)
            self.u_y = torch.tensor(u_y,dtype=torch.float32).to(device)
            self.v_force = torch.tensor(v_force,dtype=torch.float32).to(device)
        else:
            self.u_x = cp.array(u_x)
            self.u_y = cp.array(u_y)
            self.v_force = cp.array(v_force)
            self.u_x1 = torch.tensor(u_x,dtype=torch.float32).to(device)
            self.u_y1 = torch.tensor(u_y,dtype=torch.float32).to(device)
            self.v_force1 = torch.tensor(v_force,dtype=torch.float32).to(device)

    def fit_disp_init(self):
        
        if self.solver=='Adam':
            self.weights1 = torch.zeros([self.disp_model.kernel1.shape[1],2],dtype=torch.float32).to(device)
            self.weights1 = torch.nn.Parameter(self.weights1.requires_grad_())
            param=list()
            param.append(self.weights1)
            self.disp_optimizer = optim.Adam(param, lr=0.000005)
            epochs = 1000
            for epoch in range(epochs):
                xPhys = torch.ones(self.coord.shape[0], 1) * 0.5
                loss, energy_c = pinnloss2(self.weights1, self.u_x, self.u_y, self.v_force, self.problem, xPhys.to(device))
                self.disp_optimizer.zero_grad()
                loss.backward()
                self.disp_optimizer.step()
            _,u_x,u_y,v_force = cal_matrix_fast(self.disp_model, self.problem.dlX, self.problem.dlX_force, M)
            u_x = torch.tensor(u_x,dtype=torch.float32).to(device)
            u_y = torch.tensor(u_y,dtype=torch.float32).to(device)
            v_force = torch.tensor(v_force,dtype=torch.float32).to(device)
            xPhys = torch.ones(self.problem.dlX.shape[0], 1) * 0.5
            loss, energy_c = pinnloss2(self.weights1, u_x, u_y, v_force, problem, xPhys.to(device))
            self.c1 = energy_c
            self.c_0 = torch.mean(energy_c)
        else:
            xPhys_m = np.ones([self.coord.shape[0],1])*0.5
            temp = minimize(pinnloss,self.x0,jac=jac,args=(self.u_x, self.u_y, self.v_force,self.problem, xPhys_m), method='L-BFGS-B',options={'maxiter':1000})
            #temp = minimize(pinnloss,self.x0,jac=jac,tol=1e-7,args=(self.u_x, self.u_y, self.v_force,self.problem, xPhys_m, M), method='L-BFGS-B')
            self.x0 = temp.x
            _,u_x,u_y,v_force = cal_matrix_fast(self.disp_model, self.problem.dlX, self.problem.dlX_force, M)
            u_x = cp.array(u_x)
            u_y = cp.array(u_y)
            v_force = cp.array(v_force)
            xPhys_m = np.ones([self.problem.dlX.shape[0],1])*0.5
            energy_c = pinnloss3(self.x0, u_x, u_y, v_force, problem, xPhys_m)
            self.c_0 = np.mean(energy_c)

    def get_rho_new(self,total_num,xPhys,lg,cost0):
        dv = np.ones(total_num)/total_num
        dc = np.ones(total_num)
        with torch.no_grad():
            if self.solver=='Adam':
                ux = torch.matmul(self.u_x,self.weights1[:,1]).reshape(-1,1)
                uy = torch.matmul(self.u_y,self.weights1[:,1]).reshape(-1,1)
                vx = torch.matmul(self.u_x,self.weights1[:,0]).reshape(-1,1)
                vy = torch.matmul(self.u_y,self.weights1[:,0]).reshape(-1,1)   
            else: 
                ux = torch.matmul(self.u_x1,self.weights1[:,1]).reshape(-1,1)
                uy = torch.matmul(self.u_y1,self.weights1[:,1]).reshape(-1,1)
                vx = torch.matmul(self.u_x1,self.weights1[:,0]).reshape(-1,1)
                vy = torch.matmul(self.u_y1,self.weights1[:,0]).reshape(-1,1)    

            eps11 = ux
            eps12 = 0.5 * uy + 0.5 * vx
            eps22 = vy

            youngs_modulus = 1000
            poissons_ratio = 0.3
            lame_mu = youngs_modulus / (2.0 * (1.0 + poissons_ratio))
            lame_lambda = youngs_modulus * poissons_ratio / (1.0 - poissons_ratio**2)
            trace_strain = eps11 + eps22
            squared_diagonal = eps11 * eps11 + eps22 * eps22
            energy = 0.5 * lame_lambda * trace_strain * trace_strain + lame_mu * (squared_diagonal + 2.0 * eps12 * eps12)
        dc[:] = energy.squeeze().cpu().detach().numpy()
        dc = -1/2*(3*(xPhys**(3-1))*(1))*dc
        dc[:] = np.asarray(self.H*(dc[np.newaxis].T/self.Hs))[:,0]
        dv[:] = np.asarray(self.H*(dv[np.newaxis].T/self.Hs))[:,0]
        dg = dv/0.5
        (xmma,lg)=oc(total_num,xPhys,0.5,dc/cost0,dg,lg)
        xPhys[:]=np.asarray(self.H*xmma[np.newaxis].T/self.Hs)[:,0]
        return xPhys,lg

        

    def to_loss(self, coord):
        self.total_epoch = self.total_epoch+1
        xPhys_m = self.to_model(coord)
        self.xPhys = xPhys_m.squeeze().cpu().detach().numpy()
        self.lg=0
        if self.solver=='Adam':
            for i in range(5):
                self.xPhys,self.lg = self.get_rho_new(coord.shape[0], self.xPhys,self.lg, self.c_0.item())
        else:
            for i in range(5):
                self.xPhys,self.lg = self.get_rho_new(coord.shape[0], self.xPhys,self.lg, self.c_0)
        xPhys_pred = torch.tensor(self.xPhys.reshape(-1,1),dtype=torch.float32).to(device)
        alpha = min(self.problem.alpha_init + self.problem.alpha_delta * self.total_epoch, self.problem.alpha_max)
        if self.solver=='Adam':
            _, energy_c = pinnloss2(self.weights1, self.u_x, self.u_y, self.v_force, self.problem, xPhys_m)
        else:
            _, energy_c = pinnloss2(self.weights1, self.u_x1, self.u_y1, self.v_force1, self.problem, xPhys_m)
        class ComputeDeDrho(torch.autograd.Function):
            @staticmethod
            def forward(ctx, xPhys_m, energy_c, coord):
                # 将必要的模型信息保存到上下文中，以便反向传播时使用

                ctx.xPhys_m = xPhys_m
                ctx.coord = coord
                # 正向传播部分：计算能量
                #loss, energy_c = opt.pinn.pinn_loss(xPhys_m, coord)
                # 返回正向输出
                ctx.save_for_backward(xPhys_m,energy_c,coord)
                return energy_c

            @staticmethod
            def backward(ctx, denergy):
                # 获取正向传播时保存的中间结果
                xPhys_m, energy_c, coord = ctx.saved_tensors
                
                # 计算反向传播梯度
                # 使用 torch.autograd.grad 计算能量关于 xPhys_m 的梯度
                grad_energy = torch.autograd.grad(
                outputs=energy_c, 
                inputs=xPhys_m, 
                grad_outputs=denergy, 
                create_graph=True,  # 支持二阶梯度
                retain_graph=True
                )[0]
                gradients = grad_energy
                # 返回负梯度，剩余的输入梯度为零
                return -gradients, torch.zeros_like(energy_c), torch.zeros_like(coord)
        c = torch.mean(ComputeDeDrho.apply(xPhys_m, energy_c, coord))
        xPhys_dlX = self.to_model(self.problem.dlX)
        vf = torch.mean(xPhys_dlX)
        if self.solver=='Adam':
            loss =  alpha*(vf / self.problem.volfrac - 1.0) ** 2 + 1 * c / self.c_0.detach()  + 1*self.mse_loss(xPhys_pred,xPhys_m)
        else:
            loss =  alpha*(vf / self.problem.volfrac - 1.0) ** 2 + 1 * c / self.c_0 + 1*self.mse_loss(xPhys_pred,xPhys_m)
        print('Epoch:', self.total_epoch)
        print('Total Loss:', loss.item())
        print("c",c.item())
        return loss

    def fit_disp(self, epochs=200):
        if self.solver=='Adam':
            for i in range(epochs):
                #coord = self.problem.dlX_disp()
                xPhys_m = self.to_model(self.coord)
                loss, _ = pinnloss2(self.weights1, self.u_x, self.u_y, self.v_force, self.problem, xPhys_m)
                self.disp_optimizer.zero_grad()
                loss.backward()
                self.disp_optimizer.step()
        else:
            xPhys_m = self.to_model(self.coord).cpu().detach().numpy()
            temp = minimize(pinnloss,self.x0,jac=jac,args=(self.u_x, self.u_y, self.v_force,self.problem, xPhys_m), method='L-BFGS-B',options={'maxiter':epochs})
            self.x0 = temp.x
            self.weights1 = torch.tensor(np.concatenate((opt.x0[:M].reshape(-1,1),opt.x0[M:].reshape(-1,1)),axis=1),dtype=torch.float32).to(device)
            self.weights1 = torch.nn.Parameter(self.weights1.requires_grad_())
    '''
    def fit_to(self, epochs):
        
        for epoch in range(epochs):
            with torch.profiler.profile(
            activities=[
                torch.profiler.ProfilerActivity.CPU,
                torch.profiler.ProfilerActivity.CUDA  # 如果有GPU
            ],
            record_shapes=True,
            profile_memory=True,
            with_flops=True,
            with_stack=False
        ) as prof:
                self.fit_disp(50)
                if epoch % 10 == 1:
                    display.clear_output(wait=True)
        
                loss = self.to_loss(self.coord)
                self.to_optimizer.zero_grad()
                loss.backward()
                self.to_optimizer.step()
            print(prof.key_averages().table(
    sort_by="flops",  # 也可以试试 total_flops
    row_limit=20
))
    '''
    def fit_to(self, epochs):
        self.time_pde=[]
        self.time_density=[]
        for epoch in range(epochs):
            t1= time.time()
            self.fit_disp(50)
            t2 = time.time()
            self.time_pde.append(t2-t1)
            if epoch % 10 == 1:
                display.clear_output(wait=True)
            t1= time.time()
            loss = self.to_loss(self.coord)
            self.to_optimizer.zero_grad()
            loss.backward()
            self.to_optimizer.step()
            t2 = time.time()
            self.time_density.append(t2-t1)


In [ ]:
opt = DMF_TONN(problem, to_model, disp_model_h,'Adam',False)
import time
opt.fit_disp_init()

In [ ]:
t1= time.time()
opt.fit_to(400)
t2 = time.time()

Epoch: 2
Total Loss: 1.24380624294281
c 0.013174526393413544
Epoch: 3
Total Loss: 1.2871835231781006
c 0.013743173331022263
Epoch: 4
Total Loss: 1.3842377662658691
c 0.014992840588092804
Epoch: 5
Total Loss: 1.2335916757583618
c 0.01286359690129757
Epoch: 6
Total Loss: 1.2616627216339111
c 0.01320915762335062


In [9]:
xPhys_dlX = opt.to_model(opt.problem.dlXSS.to(device))
tt = np.reshape(xPhys_dlX.cpu().detach().numpy(),(2*opt.problem.nely,2*opt.problem.nelx))

In [10]:
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.linalg import spsolve

# 参数设置
nelx, nely = 120, 40       # 单元数量
E, nu = 1000, 0.3         # 杨氏模量和泊松比
rho = 0.5                 # 密度
penal = 3.0               # 惩罚因子（此处设为1，即线性）

# 平面应力条件下的单元刚度矩阵
def lk():
    E = 1000
    nu = 0.3
    k = np.array([1/2 - nu/6, 1/8 + nu/8, -1/4 - nu/12, -1/8 + 3*nu/8,
                  -1/4 + nu/12, -1/8 - nu/8, nu/6, 1/8 - 3*nu/8])
    KE = E / (1 - nu**2) * np.array([[k[0], k[1], k[2], k[3], k[4], k[5], k[6], k[7]],
                                     [k[1], k[0], k[7], k[6], k[5], k[4], k[3], k[2]],
                                     [k[2], k[7], k[0], k[5], k[6], k[3], k[4], k[1]],
                                     [k[3], k[6], k[5], k[0], k[7], k[2], k[1], k[4]],
                                     [k[4], k[5], k[6], k[7], k[0], k[1], k[2], k[3]],
                                     [k[5], k[4], k[3], k[2], k[1], k[0], k[7], k[6]],
                                     [k[6], k[3], k[4], k[1], k[2], k[7], k[0], k[5]],
                                     [k[7], k[2], k[1], k[4], k[3], k[6], k[5], k[0]]])
    return KE

KE = lk()

# 全局自由度编号
ndof = 2 * (nelx + 1) * (nely + 1)

# 节点编号
def node_ids(i, j):
    n1 = (nely + 1) * i + j
    return np.array([2 * n1, 2 * n1 + 1,
                     2 * (n1 + nely + 1), 2 * (n1 + nely + 1) + 1,
                     2 * (n1 + nely + 2), 2 * (n1 + nely + 2) + 1,
                     2 * (n1 + 1), 2 * (n1 + 1) + 1])

# 装配稀疏矩阵结构
iK = []
jK = []
sK = []
rho = tt.T[:,::-1]
for elx in range(nelx):
    for ely in range(nely):
        el = elx * nely + ely
        nodes = node_ids(elx, ely)
        rho_el = rho[elx, ely]
        for i in range(8):
            for j in range(8):
                iK.append(nodes[i])
                jK.append(nodes[j])
                sK.append((rho_el ** penal) * KE[i, j])
'''
for elx in range(nelx):
    for ely in range(nely):
        el = elx * nely + ely
        nodes = node_ids(elx, ely)
        for i in range(8):
            for j in range(8):
                iK.append(nodes[i])
                jK.append(nodes[j])
                sK.append((rho ** penal) * KE[i, j])
'''
# 生成稀疏刚度矩阵
K = coo_matrix((sK, (iK, jK)), shape=(ndof, ndof)).tocsc()

# 载荷向量
F = np.zeros(ndof)
# 右下角节点施加载荷
#frc_node = (nelx) * (nely + 1) + 0
#F[2 * frc_node + 1] = -0.1  # 垂直向下的力
# 右边中间节点施加载荷
mid_j = nely // 2
frc_node = nelx * (nely + 1) + mid_j
F[2 * frc_node + 1] = -0.1  # 垂直向下的力

# 边界条件（左边固定）
fixeddofs = []
for j in range(nely + 1):
    n = j
    fixeddofs += [2 * n, 2 * n + 1]

alldofs = np.arange(ndof)
freedofs = np.setdiff1d(alldofs, fixeddofs)

# 求解位移
U = np.zeros(ndof)
K_free = K[freedofs, :][:, freedofs]
F_free = F[freedofs]
U[freedofs] = spsolve(K_free, F_free)

# 计算 compliance
compliance = F @ U
print("Compliance:", compliance)

Compliance: 0.002521320943275929
